## HOMEWORK 4
## 1) Does basic house information reflect house's description?

In this part of the homework our task is to perform a clustering analysis of house announcements in Rome from Immobiliare.it. We will create two datasets by retrieving data from websites. Then, we will
implement two clustering methods and compare the results.

**Python Libraries:**

In [18]:
import time
import threading
import requests
from bs4 import BeautifulSoup
import re
import csv

import collect_data 

### **Scraping data**

As the first step, we are collecting the list of interesting urls, starting from the website:
https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1.

+ We are using the time.sleep function to prevent the websie blocking.
+ The implementation of "thread_find_links" function is located in the external file **collect_data.py**

In [ ]:
link_threads = []
for nr_page in range(1, 100):
    t = threading.Thread(target=collect_data.thread_find_links, args=("nr_page",))
    t.start()
    time.sleep(1)

+ We are saving collected links into **links.txt** file

In [ ]:
with open("links.txt", "w") as link_file:
    for url_ in link_threads:
        link_file.write(url_)

In [2]:
# Load data from link.txt 
with open("links.txt", "r") as link_file:
    link_threads = link_file.readlines()

+ We are iterating through collected links to retrieve and save data about flats. 
+ For this, we defined the fuction **threadExtract**. 
+ We applied the parallel programming by generating threads for each url. 

In [31]:
def threadExtract(url):
    """
    This function gets html content and scraps the interesting data
    Input: String: "url" 
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    scrapped_data = collect_data.scrap_data(soup)
    if scrapped_data != None:
        collected_data.append(scrapped_data)
        datawriter.writerow(scrapped_data)

+ The collected data are saved in **flat_data.txt** file.

In [35]:
data_file = open("flat_data.txt", "w", encoding='utf-8')
headers = ["price", "locali", "superficie", "bagni", "piano", "description"]
datawriter = csv.writer(data_file)
datawriter.writerow(headers)

49

In [36]:
count = 0
failed_links = []
collected_data = []
for link_ in link_threads:
    try:
        threading.Thread(threadExtract(link_[:-1])).start() #-1 because we have "\n" in the end of a link
        time.sleep(2) #for non-blocking
        count += 1
        if count >= 10000:break
    except Exception as e:
        print(e)
        failed_links.append(link_)
        
data_file.close()

+ Reading the **flat_data.txt** file content as dataframe:

In [ ]:
data_file = open("flat_data.txt", "r", encoding='utf-8')